In [60]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
!dir

imgs  labels  labels_with_text


#Преобразование изображений и разметки в табличный вид

In [63]:
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Путь к папкам
images_dir = 'imgs'  # Папка с изображениями
labels_dir = 'labels'  # Папка с разметками
labels_text_dir = "labels_with_text"

# Список для хранения данных таблицы
data = []

# Обход папки с изображениями
for image_file in os.listdir(images_dir):
    if image_file.endswith(('.png', '.jpg', '.jpeg', '.JPG')):
        image_path = os.path.join(images_dir, image_file)
        label_path = os.path.join(labels_dir, os.path.splitext(image_file)[0] + '.txt')
        label_text_path = os.path.join(labels_text_dir, os.path.splitext(image_file)[0] + '.bbox')

        # Проверка наличия файла разметки
        if os.path.exists(label_path):


            # Чтение текстовых меток
            if os.path.exists(label_text_path):
                with open(label_text_path, 'r') as f:
                    label_text = ''.join(f.readlines())
                    label_text = label_text.replace('\n', '')
            else:
                label_text = np.NaN


            # Чтение текстовых меток
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    label = ''.join(f.readlines())
            else:
                label = np.NaN

            # Добавление данных в таблицу
            data.append([image_file, label, label_text])

# Создание DataFrame с изображениями и текстовыми метками
df = pd.DataFrame(data, columns=['image_file', 'label', 'label_text'])



In [64]:
df

,image_file,label,label_text
0,12.JPG,0 0.580977 0.334656 0.277530 0.419312\n,"""1753-30-1013 117"""
1,1.JPG,0 0.659722 0.350446 0.414683 0.040179\n0 0.795...,"""АНЕМ.492664.500-03LP 497/2"""
2,102.JPG,0 0.436012 0.569114 0.496032 0.793651\n,"""АМ117.07.01.211 29"""
3,120.JPG,0 0.587302 0.509673 0.791005 0.470238\n,"""АМ117.07.01.220 13"""
4,299.jpg,0 0.477766 0.447012 0.378501 0.285247\n,"""АМ116.06.00.901-03 1"""
...,...,...,...
248,96.JPG,0 0.534375 0.399609 0.737500 0.661719\n,"""АМ117.07.01.211 26"""
249,92.jpg,0 0.476797 0.380637 0.692157 0.123039\n,"""1391-30-1151"""
250,94.JPG,0 0.576562 0.473828 0.726042 0.666406\n,"""АМ117.07.01.211 25"""
251,83.jpg,0 0.491830 0.744118 0.509150 0.113725\n0 0.251...,"""195-30-1221 3422-3433"""


#Создание submission.csv и grounded_true.csv

In [65]:
gt = df

In [66]:
import pandas as pd
import numpy as np

def simul_sub(df, columns_to_swap_modify):
    len_base_df = len(df)

    # Процент строк для удаления и дублирования
    fraction_to_remove = 0.5
    fraction_to_duplicate = 0.1

    num_to_remove = int(len_base_df * fraction_to_remove)
    num_to_duplicate = int(len_base_df * fraction_to_duplicate)

    # Удаление случайных строк
    rows_to_remove = df.sample(num_to_remove).index
    data_dropped = df.drop(rows_to_remove)

    # Дублирование случайных строк
    rows_to_duplicate = data_dropped.sample(num_to_duplicate)
    data_final = pd.concat([data_dropped, rows_to_duplicate])
    data_final = data_final.reset_index(drop=True)

    # Замена значений с вероятностью swap_chance
    swap_chance=0.3

    for idx in data_final.index:
        for column in columns_to_swap_modify:
            if np.random.rand() < swap_chance:
                # Выбираем случайное значение из того же столбца
                random_row = data_final.sample(1).iloc[0]
                data_final.at[idx, column] = random_row[column]

    return data_final


In [67]:
sub = simul_sub(gt.copy(), ['label', 'label_text'])

In [68]:
#sub = gt.copy()
#sub = sub.iloc[:2]

#Метрика

score = Средний IoU рамок * 0.05 + Средняя посимвольная точность текста * 0.65 + Точность абсолютно верного распознавания текста *0.3

In [69]:
!pip install Levenshtein

In [70]:
from shapely.geometry import box
from shapely.ops import unary_union
from Levenshtein import ratio
import pandas as pd
import numpy as np

# Функция для преобразования строки label в список bbox
def parse_label(label):
    try:
        boxes = []
        for item in label.split('\n'):
            if item:
                _, x_center, y_center, width, height = map(float, item.split())
                boxes.append([
                    x_center - width / 2, y_center - height / 2,
                    x_center + width / 2, y_center + height / 2
                ])
        return boxes
    except:
        raise Exception("Неправильный формат строки 'label'. Ожидается формат: '0 x_center y_center width height\\n...'")

# Функция для вычисления совокупного IoU для нескольких bbox
def calculate_total_iou(predicted_boxes, true_boxes):
    predicted_polygons = [box(*pred_box) for pred_box in predicted_boxes]
    true_polygons = [box(*true_box) for true_box in true_boxes]
    predicted_union = unary_union(predicted_polygons)
    true_union = unary_union(true_polygons)
    intersection_area = predicted_union.intersection(true_union).area
    union_area = predicted_union.union(true_union).area
    total_iou = intersection_area / union_area if union_area != 0 else 0
    return total_iou

# Основная функция для расчета метрик
def calc_metrics(ground_truth, submission):
    ious = []
    character_accuracies = []
    complete_matches = []

    ground_truth['text_is_notna'] = ground_truth['label_text'].notna()

    for _, row in ground_truth.iterrows():
        image_file = row['image_file']
        sub_row = submission[submission['image_file'] == image_file]

        if not sub_row.empty:
            gt_boxes = parse_label(row['label'])
            pred_boxes = parse_label(sub_row.iloc[0]['label'])
            iou = calculate_total_iou(pred_boxes, gt_boxes)
            ious.append(iou)

            if row['text_is_notna']:
                char_acc = ratio(row['label_text'], sub_row.iloc[0]['label_text'])
                character_accuracies.append(char_acc)
                complete_matches.append(row['label_text'] == sub_row.iloc[0]['label_text'])

    # Заполняем отсутствующие значения
    ious.extend([0] * (len(ground_truth) - len(ious)))
    character_accuracies.extend([0] * (ground_truth['text_is_notna'].sum() - len(character_accuracies)))
    complete_matches.extend([0] * (ground_truth['text_is_notna'].sum() - len(complete_matches)))

    mean_iou = np.mean(ious)
    mean_character_accuracy = np.mean(character_accuracies)
    accuracy = np.mean(complete_matches)

    return {
        "Средний IoU рамок": mean_iou,
        "Средняя посимвольная точность текста": mean_character_accuracy,
        "Точность абсолютно верного распознавания текста": accuracy
    }

# Расчет финального балла
def calc_score(metrics):
    score = (metrics["Средний IoU рамок"] * 0.05 +
             metrics["Средняя посимвольная точность текста"] * 0.65 +
             metrics["Точность абсолютно верного распознавания текста"] * 0.3)
    return score


In [71]:
metrics = calc_metrics(gt, sub)
metrics

{'Средний IoU рамок': 0.39177233019443236,
 'Средняя посимвольная точность текста': 0.4292842384429139,
 'Точность абсолютно верного распознавания текста': 0.35177865612648224}

In [72]:
score = calc_score(metrics)
score

0.4041569683355603